In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [47]:
analyst_data = pd.read_csv('data/analyst_consensus_data.csv')

In [48]:
analyst_data.head()

,Ticker,AveragePriceTarget,NumRatings,ClosingPrice
0,A,162.285714,14,138.210007
1,AA,53.000000,11,40.340000
2,AADI,37.333333,3,7.320000
3,AAL,17.285714,14,14.160000
4,AAN,14.300000,5,9.620000


In [49]:
# Making AveragePriceTarget, ClosingPrice rounded to 2 places
analyst_data['AveragePriceTarget'] = analyst_data['AveragePriceTarget'].round(2)
analyst_data['ClosingPrice'] = analyst_data['ClosingPrice'].round(2)

In [50]:
analyst_data['ExpectedGrowth%'] = ((analyst_data['AveragePriceTarget']-analyst_data['ClosingPrice'])/analyst_data['ClosingPrice']).round(3)

In [51]:
# Removing an Outlier
analyst_data.drop([700], inplace = True)

In [52]:
analyst_data[analyst_data['ExpectedGrowth%'] == 212.333]

,Ticker,AveragePriceTarget,NumRatings,ClosingPrice,ExpectedGrowth%
673,CFRX,320.0,2,1.5,212.333


In [53]:
# Removing Rows that have less than 4 ratings
analyst_data = analyst_data[analyst_data['NumRatings'] >= 3]

In [54]:
analyst_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3268 entries, 0 to 3465
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Ticker              3268 non-null   object 
 1   AveragePriceTarget  3268 non-null   float64
 2   NumRatings          3268 non-null   int64  
 3   ClosingPrice        3267 non-null   float64
 4   ExpectedGrowth%     3267 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 153.2+ KB
